In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import geopy.distance
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score


In [9]:
conflict_df = pd.read_csv('../data/conflict_data_irq.csv')
conflict_df.drop(0, inplace=True)
conflict_df['date_start'] = pd.to_datetime(conflict_df['date_start'])
conflict_df['date_end'] = pd.to_datetime(conflict_df['date_start'])

In [10]:
governorate_dict = {'Al Anbār province':'Anbar', 'Nīnawá province':'Ninewa',
'Baghdād province':'Baghdad','Dahūk province':'Dahuk', 'Diyālá province':'Diyala','Kirkūk province':'Kirkuk',
'Şalāḩ ad Dīn province':'Salah al-Din', 'Karbalā’ province':'Kerbala', 'An Najaf province':'Najaf',
'Bābil province':'Babylon','Arbīl province':'Erbil','Al Başrah  province':'Basrah','Dhī Qār province':'Thi-Qar',
'Al Muthanná province':'Muthanna','As Sulaymānīyah province':'Sulaymaniyah', 'Maysān  province':'Missan',
'Al Qādisīyah province':'Qadissiya', 'Wāsiţ province':'Wassit'}

In [11]:
conflict_df.replace(to_replace=governorate_dict, inplace=True)

In [12]:
district_dict = {'Abū Ghurayb district':'Abu Ghraib',
'Al Ba‘āj district':"Al-Ba'aj",
'Al Qā’im district':"Al-Ka'im",
'Al-Faris district':'Al-Fares',
'Hīt district':'Heet',
'Qaḑā’ ‘Ānah':'Ana',
'Qaḑā’ ad Dawr':'Al-Daur',
'Qaḑā’ al Fallūjah':'Falluja',
'Qaḑā’ al Ḩaḑr':'Hatra',
'Qaḑā’ al Ḩamdānīyah':'Al-Hamdaniya',
'Qaḑā’ al Khāliş':'Al-Khalis',
'Qaḑā’ al Maḩmūdīyah':'Mahmoudiya',
'Qaḑā’ al Mawşil':'Mosul',
'Qaḑā’ al Miqdādīyah':'Al-Muqdadiya',
'Qaḑā’ ar Ramādī':'Ramadi',
'Qaḑā’ ar Ruţbah':'Al-Rutba',
'Qaḑā’ ash Shaykhān':'Al-Shikhan',
'Qaḑā’ Balad':'Balad',
'Qaḑā’ Bayjī':'Baiji',
'Qaḑā’ Haditha':'Haditha',
'Qaḑā’ Khānaqīn':'Khanaqin',
'Qaḑā’ Kifrī':'Kifri',
'Qaḑā’ Sāmarrā':'Samarra',
'Qaḑā’ Sharqāţ':'Al-Shirqat',
'Qaḑā’ Tall ‘Afar':'Telafar',
'Qaḑā’ Tikrīt':'Tikrit',
'Qaḑā’ Zākhū':'Zakho',
'Sinjār district':'Sinjar',
'Tallkayf district':'Tilkaif',
'Tooz district':'Tuz Khurmatu',
'Zakho district':'Zakho',
"Al ‘Amādīyah":'Amedi',
"Al ‘Amādīyah district":'Amedi',
"Al ‘Azīzīyah district":'Al-Azezia',
"Al Madā’in district":'Al-Midaina',
'Al Majar al Kabīr district':'Al-Mejar Al-Kabir',
'Al Maymūnah district':'Al-Maimouna',
'Al-Shirqat district':'Al-Shirqat',
'Baladrūz district':'Baladrooz',
'Dāqūq district':'Daquq',
'Kuwaysinjaq district':'Koisnjaq',
'Mergasur District':'Mergasur',
'Paynjuwayn district':'Penjwin',
"Qaḑā al ‘Amārah":'Amara',
'Qaḑā al Chibāyish':'Al-Chibayish',
'Qaḑā Karbalā’':'Kerbala',
"Qaḑā’ ‘Alī al Gharbī":'Ali Al-Gharbi',
'Qaḑā’ ad Dīwānīyah':'Diwaniya',
'Qaḑā’ Ain Al Tamur':'Ain Al-Tamur',
'Qaḑā’ al Başrah':'Basrah',
'Qaḑā’ al Hāshimīyah':'Hashimiya',
'Qaḑā’ al Ḩawījah':'Al-Hawiga',
'Qaḑā’ al Ḩayy':'Al-Hai',
'Qaḑā’ al Ḩillah':'Hilla',
'Qaḑā’ al Hindīyah':'Al-Hindiya',
'Qaḑā’ al Kūfah':'Kufa',
'Qaḑā’ al Kūt':'Kut',
'Qaḑā’ al Maḩāwīl':'Al-Mahawil',
'Qaḑā’ al Manādhirah':'Al-Manathera',
'Qaḑā’ al Musayyib':'Al-Musayab',
'Qaḑā’ al Qurnah':'Al-Qurna',
'Qaḑā’ an Najaf':'Najaf',
'Qaḑā’ an Nāşirīyah':'Nassriya',
'Qaḑā’ Arbīl':'Erbil',
'Qaḑā’ as Samāwah':'Al-Samawa',
'Qaḑā’ aş Şuwayrah':'Al-Suwaira',
'Qaḑā’ ash Shāmīyah':'Al-Shamiya',
'Qaḑā’ ash Shaţrah':'Al-Shatra',
'Qaḑā’ az Zubayr':'Al-Zubair',
"Qaḑā’ Ba‘qūbah":"Ba'quba",
'Qaḑā’ Chamchamal':'Chamchamal',
'Qaḑā’ Chomān':'Choman',
'Qaḑā’ Dahūk':'Dahuk',
'Qaḑā’ Dibis':'Dabes',
'Qaḑā’ Ḩadīthah':'Haditha',
'Qaḑā’ Ḩalabchah':'Halabja',
'Qaḑā’ Kalār':'Kalar',
'Qaḑā’ Kirkūk':'Kirkuk',
'Qaḑā’ Makhmūr':'Makhmur',
'Qaḑā’ Miqdādīyah':'Al-Muqdadiya',
'Qaḑā’ Pishdar':'Pshdar',
'Qaḑā’ Rāniyah':'Rania',
'Qaḑā’ Shahrbāzār':'Sharbazher',
'Qaḑā’ Shaqlāwah':'Shaqlawa',
'Qaḑā’ Sulaymānīyah':'Sulaymaniya',
'Qaḑā’ Sūq ash Shuyūkh':'Suq Al-Shoyokh',
'Qaḑā’ Zākhū':'Zakho',
'Soran district':'Soran'}

In [13]:
conflict_df.replace(to_replace=district_dict, inplace=True)

In [15]:
conflict_df['adm_2']

1              Falluja
2       Qaḑā’ Sāmarrā’
3                Baiji
4              Falluja
5                Balad
             ...      
7469             Soran
7470             Zakho
7471               NaN
7472             Mosul
7473           Telafar
Name: adm_2, Length: 7473, dtype: object

In [18]:
district

AttributeError: 'builtin_function_or_method' object has no attribute 'values'

In [22]:
conflict_df = conflict_df[conflict_df['adm_2'].isin(list(district_dict.values()))]

In [ ]:
%%time
#reads in data on outflow of refugees, concatenates into big dataframe
outflow_filepaths = [s for s in listdir("../data/out/")]
out_df = pd.concat((pd.read_csv("../data/out/"+s) for s in outflow_filepaths), ignore_index=True) 


In [ ]:
out_df['Governorate'].unique()

In [ ]:
out_df.dropna(how='any', inplace=True)

In [ ]:
#cleanup of data types 
out_df['date'] = pd.to_datetime(out_df['date'])
out_df.rename(columns={'Location Name':'Location_name'}, inplace=True)
out_df['Location ID'] = out_df['Location ID'].astype(int)
out_df['Place ID'] = out_df['Place ID'].astype(int)
out_df['Families'] = out_df['Families'].astype(int)
out_df['Individuals'] = out_df['Individuals'].astype(int)


In [ ]:
#filters only for districts on which there exists return data 
out_df = out_df[out_df['District'].isin(district_names)]

In [ ]:
%%time
#reads in data on returning refugees, concatenates into large dataframe 
returnee_filepaths = [f for f in listdir("../data/inflow/")]
ret_df = pd.concat((pd.read_csv("../data/inflow/"+f)
                  for f in returnee_filepaths), ignore_index=True)

In [ ]:
ret_df

In [ ]:
diplacement_dict = {'Pre June14 Period of displacement':'disp_preJun14',
                      'June July14 Period of displacement':'disp_JunJuly14',
                      'August14 Period of displacement':'disp_Aug14',
                      'Post September 14 Period of displacement':'disp_postSep14',
                      'Post April15 Period of displacement':'disp_postApr15',
                      'Post March 16 Period of displacement':'disp_postMar16',
                      'Post 17 October 16 Period of displacement': 'disp_post17Oct16',
                      'July 17 Period of displacement':'disp_Jul17',
                      'Jan19':'disp_Jan19'}

In [ ]:
ret_df.rename(columns=displacement_dict, inplace=True)
out_df.rename(columns=diplacement_dict, inplace=True)

In [ ]:
ret_df.dropna(how='all', inplace=True)
ret_df['date'] = pd.to_datetime(ret_df['date'])

In [ ]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)



In [ ]:
ret_df = trim_all_columns(ret_df)
out_df = trim_all_columns(out_df)

In [ ]:
ret_df.sort_values(['Location ID', 'date'], inplace=True)

In [ ]:
ret_df.dropna(how='any', inplace=True)

In [ ]:
ret_df['ret_delta'] = ret_df.groupby(['Location ID'])['Returnee Families'].transform(lambda x: x.diff()) 



In [ ]:
ret_df.sort_values(['Location ID', 'date'], inplace=True)
ret_df.reset_index(inplace=True)
ret_df.drop(columns='index', axis=1, inplace=True)

In [ ]:
ret_df.columns

In [ ]:
#should be functionized 
for i in range(len(ret_df)):
    if np.isnan(ret_df.at[i, 'ret_delta']):
        ret_df.at[i, 'ret_delta'] = ret_df.at[i, 'Returnee Families']
    else:
        None

In [ ]:
out_df.sort_values(['Location ID', 'date'], inplace=True)
out_df.reset_index(inplace=True)
out_df.drop(columns='index', axis=1, inplace=True)

In [ ]:
master_df = ret_df.merge(out_df, how='outer', on=['Location ID', 'date', 'Governorate',
                                                 'District', 'Location_name'])

In [ ]:
master_df.rename(columns={'Families':'outflow', 'ret_delta':'inflow'}, inplace=True)

In [ ]:
master_df.columns

In [ ]:
master_df['date'] = master_df['date'].apply(lambda x: x.toordinal())
master_df.fillna(0, inplace=True)

In [ ]:
X = master_df.drop(columns=['inflow'])
y = master_df['inflow']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

In [ ]:
print("accuracy score:", rf.score(X_test, y_test))

In [ ]:

print("\n10. precision:", precision_score(y_test, y_predict))
print("    recall:", recall_score(y_test, y_predict))



In [ ]:
for ix, coord in enumerate(zip(conflict_df['latitude'], conflict_df['longitude'])):
    for idx, rcoord in enumerate(zip(ret_df['Latitude'], ret_df['Longitude'])):
        if geopy.distance.distance(coord, rcoord).km > 10:
            print(idx)
            ser[idx] += 1
        else:
            print(idx)